In [2]:
import tensorflow as tf
from pathlib import Path
import numpy as np

# ── 1) Define a Soft‐Voting Ensemble Model ─────────────────────────────────
class SoftVotingEnsemble(tf.keras.Model):
    def __init__(self, member_paths):
        super().__init__()
        # Load each fold‐model
        self.members = [tf.keras.models.load_model(str(p)) for p in member_paths]

    @tf.function(input_signature=[tf.TensorSpec([None, 512, None], tf.float32)])
    def call(self, inputs):
        # Collect each member’s probabilities
        preds = [m(inputs, training=False) for m in self.members]
        # Average over the first axis (models)
        return tf.reduce_mean(tf.stack(preds, axis=0), axis=0)

# ── 2) Paths & Top‐3 Selection ──────────────────────────────────────────────
base_dir   = Path.cwd().parent
model_dir  = base_dir / "data" / "trained_models"
output_dir = base_dir / "results" / "ensembles"
output_dir.mkdir(parents=True, exist_ok=True)


EXPERIMENTS = {
    "baseline_raw"   : ["raw"],
    "raw+water"      : ["raw","water"],
    "raw+fits"       : ["raw","fit1","fit2"],
    "all_four"       : ["raw","water","fit1","fit2"],
    "fits+water"     : ["water","fit1","fit2"],
    "fit1_only"      : ["fit1"],
    "fit2_only"      : ["fit2"],
    "fits_combined"  : ["fit1", "fit2"],
}

# Suppose `top3_keys` is your list of the three best experiment names:
top3_keys = ['fits_combined', 'fits+water', 'all_four']


# ── 3) Instantiate & Save Each Ensemble ────────────────────────────────────
for exp_key in top3_keys:
    # Paths to the 5 fold models
    member_paths = [model_dir / f"{exp_key}_tuned_fold{fold}" for fold in range(1, 6)]
    # Build the ensemble model
    ensemble_model = SoftVotingEnsemble(member_paths)
    # suppose for this experiment you have n_ch channels:
    channels = EXPERIMENTS[exp_key]
    n_ch = len(channels)
    # 2) bind the model to an input shape
    ensemble_model.build(input_shape=(None, 512, n_ch))
    # Save as TensorFlow SavedModel
    save_path = output_dir / f"{exp_key}_ensemble"
    tf.keras.models.save_model(ensemble_model, str(save_path), include_optimizer=False)
    print(f"Saved ensemble model for '{exp_key}' to:\n  {save_path}")


INFO:tensorflow:Assets written to: e:\ArtifactRemovalProject\results\ensembles\fits_combined_ensemble\assets


INFO:tensorflow:Assets written to: e:\ArtifactRemovalProject\results\ensembles\fits_combined_ensemble\assets


Saved ensemble model for 'fits_combined' to:
  e:\ArtifactRemovalProject\results\ensembles\fits_combined_ensemble


INFO:tensorflow:Assets written to: e:\ArtifactRemovalProject\results\ensembles\fits+water_ensemble\assets


INFO:tensorflow:Assets written to: e:\ArtifactRemovalProject\results\ensembles\fits+water_ensemble\assets


Saved ensemble model for 'fits+water' to:
  e:\ArtifactRemovalProject\results\ensembles\fits+water_ensemble


INFO:tensorflow:Assets written to: e:\ArtifactRemovalProject\results\ensembles\all_four_ensemble\assets


INFO:tensorflow:Assets written to: e:\ArtifactRemovalProject\results\ensembles\all_four_ensemble\assets


Saved ensemble model for 'all_four' to:
  e:\ArtifactRemovalProject\results\ensembles\all_four_ensemble
